Hello World!

In [174]:
from skopt import gp_minimize #Doc here: https://scikit-optimize.github.io/#skopt.gp_minimize
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.api import QPUConnection
from pyquil.gates import X, Z, I, MEASURE
from pyquil.paulis import PauliTerm, sZ, check_commutation, exponentiate_commuting_pauli_sum, exponentiate, sI
from Universe import Universe
import numpy as np

qvm = QVMConnection()

Section I: Initialization
------------------------

We flip one qubit in each logical group on, corresponding to the ordering 1 --> 2 --> 3 --> 4:

In [ ]:
initialization = Program(X(0), X(5), X(10), X(15))

We also define what the partition for our cities and ordering indeces will be. In this section we use the *color-parity partition* defined on page 20 of this [QAOA Heuristic paper][2] to partition the Hamiltonian terms into a mutually commuting set of unitary operators. First we define the partition (for the **n=4** case):

[2]: https://arxiv.org/pdf/1709.03489.pdf

In [77]:
city_partitions = [[[0,1],[2,3]],[[0,2],[1,3]],[[0,3],[1,2]]] # Each 2-tuple correspnds to an edge (u,v)
order_partitions = [[0,2],[1,3]] # Each number corresponds to an possible position in the ordering (i)
num_qubits = 4**2

random = np.random.uniform(low = 1, high = 20, size=[4,4])
distance_array = np.floor((random + np.transpose(random))/2)

Section II: Defining the Cost Hamiltonian (Phase Separator)
-------------------------------------------------------

In [138]:
def z(city_index, row_index):
    return sZ((city_index * 4 + row_index) % num_qubits )
    
def weight(city_1, city_2, row_index):
    return PauliTerm('I', (city_1 * 4 + row_index) % num_qubits, distance_array[city_1, city_2])

def H_cost_term(i, u, v):
    return  weight(u, v, i) * z(u, i) * z(v , i + 1)

def H_cost_partition(city_partition, order_partition):
    result = 0
    for uv in city_partition:
        for i in order_partition:
            result += H_cost_term(uv[0],uv[1],i)           
    return result


def U_cost_partition(city_partition, order_partition):
    city = city_partition[0][0]
    index = order_partition[0]
    
    weight = PauliTerm('I', (city * 4 + index) % num_qubits )
    
    H_cost = H_cost_partition(city_partition, order_partition)
    
    comm_count = True
    
    for element in H_cost.terms:
        comm_count *= check_commutation(H_cost.terms, element)
    
    if comm_count == True:
        result = exponentiate_commuting_pauli_sum(H_cost + weight)
        return result
    
    else:
        result = exponentiate(H_cost) * weight
        return result
   
def U_cost_all(city_partitions, order_partitions, gamma):
    result = Program(I(0))

    for city_partition in city_partitions:
        for order_partition in order_partitions:
            unitary = U_cost_partition(city_partition, order_partition)
            
            result = result.inst(unitary(gamma))
            
    return result

Section III: Defining the Driver Hamiltonian (Mixer)
-------------------------------------------------

In [159]:
from pyquil.paulis import exponentiate, PauliSum, PauliTerm, check_commutation, exponentiate_commuting_pauli_sum, commuting_sets

# hardcoded to 4 for now
def get_qubit(city, order, num_cities=4):
    return city * num_cities + order


def S_plus(qubit):
    '''
    S+ operation used in mixing hamiltonian for the Travelling Salesperson Problem found in arXiv:1709.03489

    :param qubit: qubit to apply S+ operation to
    :return:
    '''
    return PauliTerm("X", qubit, 1) + PauliTerm("Y", qubit, complex(0, 1))


def S_minus(qubit):
    '''
    S- operation used in mixing hamiltonian for the Travelling Salesperson Problem found in arXiv:1709.03489

    :param qubit: qubit to apply S- operation to
    :return:
    '''
    return PauliTerm("X", qubit, 1) + PauliTerm("Y", qubit, complex(0, -1))


def H_iuv(i, u, v):
    '''
    generates a partial mixing hamiltonian for the Travelling Salesperson Problem found in arXiv:1709.03489

    :param i: order term (e.g. the ith stop)
    :param u: source city
    :param v: destination city
    :return:
    '''
    qubit_ui = get_qubit(u, i)
    qubit_vi1 = get_qubit(v, i+1)
    qubit_ui1 = get_qubit(u, i + 1)
    qubit_vi = get_qubit(v, i)

    terms = [
        S_plus(qubit_ui) * S_plus(qubit_vi1) * S_minus(qubit_ui1) * S_minus(qubit_vi),
        S_minus(qubit_ui) * S_minus(qubit_vi1) * S_plus(qubit_ui1) * S_plus(qubit_vi)
    ]

    result = terms[0] + terms[1]

    return result

def H_mixer_partition(city_partition, order_partition):
    result = 0
    for uv in city_partition:
        for i in order_partition:
            result += H_iuv(i,uv[0],uv[1])          
    return result


def U_partition(city_partition, order_partition):
    '''
    partial color parity mixer for the Travelling Salesperson Problem found in arXiv:1709.03489

    :param B:
    :param city_partitions: list of city partitions (which are lists themselves of tuples of cities).
                            Equivalent to the color partitions in the original algorithm
    :param order_partitions: list of order partitions (which are lists themselves of orders).
                            Equivalent to the parity partitions in the original algorithm
    :return:
    '''
    H_mixer = H_mixer_partition(city_partition, order_partition)
    
    comm_count = True
    
    for element in H_mixer.terms:
        comm_count *= check_commutation(H_mixer.terms, element)
    
    if comm_count == True:
        result = exponentiate_commuting_pauli_sum(H_mixer)
        return result
    
    else:
        result = exponentiate(H_mixer) * weight
        return result


def color_parity_mixer(B, city_partitions, order_partitions):
    '''
    implements the full color parity mixer for the Travelling Salesperson Problem found in arXiv:1709.03489

    :param B:
    :param city_partitions: list of city partitions (which are lists themselves of tuples of cities).
                            Equivalent to the color partitions in the original algorithm
    :param order_partitions: list of order partitions (which are lists themselves of orders).
                            Equivalent to the parity partitions in the original algorithm
    :return:
    '''
    result = Program(I(0))

    for order_partition in order_partitions:
        for city_partition in city_partitions:
            unitary = U_partition(city_partition, order_partition)
            result = result.inst(unitary(B))
            
    return result

Section IV: Helper Methods For Bayesian Optimization
-------------------------------------------------

In [163]:
def neg_cost_vector(qaoa_output, num_cities=4):
    trials = qaoa_output.shape[0]

    if qaoa_output.shape[1] != num_cities^2:
        ValueError("The input samples have the wrong number of cities")

    cost_vector = np.empty(trials)

    for trial in range(trials):
        qaoa_trial = qaoa_output[trial]
        cities_order = np.empty(num_cities)

        cost = 0

        for n in range(num_cities):
            cities_order[n] = np.nonzero(qaoa_trial[n: n + (num_cities - 1)])[0]

            city_1 = cities_order[n % num_cities]
            city_2 = cities_order[(n + 1) % num_cities]
            cost += map.get_distance(city_1, city_2)

        cost_vector[trial] = cost

    return - 1.0 *cost_vector


def first_order_mean(neg_cost_vector):
    '''
    :param cost_vector: a vector containing samples from the cost distribution
    :return: Montecarlo estimate of the first order statistic based on cost distribution
    '''

    trials = len(neg_cost_vector)
    first_order_trial = np.empty(trials)


    for n in range(trials):
        cum_prob_n = np.sum(np.less_equal(neg_cost_vector, neg_cost_vector[n] * np.ones(trials)))/float(trials)
        first_order_trial = neg_cost_vector[n] * (1- cum_prob_n)**(trials - 1)

    first_order_mean = np.sum(first_order_trial)

    return first_order_mean


def first_order_actual(neg_cost_vector):
    return np.min(neg_cost_vector)

Section IV: Combining Mixer and Driver Unitaries
---------------------------------------------

In [191]:
def target_function(gamma, beta, initialization, city_partitions, order_partitions):
    master_program_phase = initialization.inst(U_cost_all(city_partitions, order_partitions, gamma))
    master_program = master_program_phase.inst(color_parity_mixer(beta, city_partitions, order_partitions))
    
    results = qvm.run_async(master_program, classical_addresses = range(17), trials = 1)
    
    #Function not complete need to include calculation of actual target function based on qvm results
    return results

In [192]:
print(target_function(1.0, 1.0, initialization, city_partitions, order_partitions))

23925
GCDFhgxvWVvAbEeM
